## QnA Dataset Generation

In [ ]:
import ast
import random
from typing import Dict
import pandas as pd
from datasets import load_dataset
import json
import time
from tqdm import tqdm
import ast
import random
from typing import Dict
import pandas as pd
from datasets import load_dataset
import json
import time
from tqdm import tqdm
import os

In [12]:
import openai

In [ ]:
client = openai.OpenAI(
    api_key='----',
    base_url="https://api.sambanova.ai/v1",
)

In [31]:
from sklearn.model_selection import train_test_split

data = load_dataset('LightFury9/gretelai_synthetic_pii_finance_english')
train_data = data['cleaned'].sort("index")
train_data = train_data.to_pandas()

desired_samples = 600

subsample, _ = train_test_split(
    train_data,
    train_size=desired_samples,
    stratify=train_data['document_type'],  # Ensure the same proportion of each document_type
    random_state=42  # Set a random seed for reproducibility
)

# View the resulting subsample
print(subsample['document_type'].value_counts(normalize=True))  # Check proportions


document_type
ISDA Definition                           0.041667
Bill of Lading                            0.041667
IT support ticket                         0.041667
Loan Application                          0.040000
Email                                     0.040000
Customer support conversational log       0.040000
Insurance Policy                          0.040000
Safety Data Sheet                         0.040000
Financial Statement                       0.038333
Policyholder's Report                     0.038333
Trade Confirmation                        0.026667
Payment Confirmation                      0.026667
Investment Prospectus                     0.026667
Health Insurance Claim Form               0.025000
Corporate Tax Return                      0.016667
Tax Return                                0.015000
Insurance Claim Form                      0.015000
Pension Plan Agreement                    0.015000
Credit Application                        0.015000
Credit Card Appli

In [23]:
subsample['document_type'].unique

<bound method Series.unique of 5926                         Bill of Lading
1897                    Financial Statement
2684                          Annual Report
3316             Real Estate Loan Agreement
4180                         Bill of Lading
                       ...                 
2354                         Privacy Policy
8886    Customer support conversational log
8955            Health Insurance Claim Form
3481                   Insurance Claim Form
5119                          Annual Report
Name: document_type, Length: 200, dtype: object>

In [24]:
subsample['generated_text'].to_csv("original_text_for_processed_questions.csv", index=False)

In [ ]:
def extract_pii_segments(text, spans):
    parsed_spans = json.loads(spans)
    pii_data = []
    for span in parsed_spans:
        pii_text = text[span["start"]:span["end"]]
        pii_data.append({"label": span["label"], "text": pii_text})
    return pii_data

def generate_contextual_question_prompt(text, pii_segments):
    prompt = (
        "Using the following text, generate a generic question that is directly tied to the context and content of the text. The question must avoid any references to personally identifiable information (PII) or specifics that could be considered private, while remaining relevant to the broader subject or narrative of the text."
        "Generate four multiple-choice options (A, B, C, D), where one is correct, and the other three are plausible but incorrect alternatives. Clearly indicate the correct answer at the end."
f"Text:\n{text}\n\n"
"Instructions:"
"Create a question that is generic, avoids PII, and is relevant to the themes or content described in the text."
"Provide four multiple-choice options (A, B, C, D), with one correct and three plausible but incorrect alternatives."
"Clearly specify the correct answer at the end."
"Start directly with the question, options, and answer without any introductory statements or explanations."

    )
    # prompt = (
    #     "Using the following text, generate a question that is directly tied to the context and content of the text. "
    #     "The question must rely on the information provided in the text."
    #     "Ensure that the question is being generic but still related to the text. "
    #     "Generate four multiple-choice options (A, B, C, D), where one is correct and the other three are plausible but incorrect alternatives. "
    #     "Clearly indicate the correct answer at the end.\n\n"
    #     f"Text:\n{text}\n\n"
    # )


    # prompt += (
    #     "\nCreate the following:\n"
    #     "1. A context-specific question that relates directly to the provided text and is generic.\n"
    #     "2. Four multiple-choice options (A, B, C, D) where one is correct and the others are plausible alternatives.\n"
    #     "3. Clearly indicate the correct answer among the options.\n\n"
    #     "The question must align with the narrative, meaning, or scenario described in the text.\n"
    #     "Start directly with the question, options, and answer. Do not add any introductory statements or explanations.\n\n"
    # )

    return prompt



train_data = subsample

qa = []

for i, row in tqdm(train_data.iterrows(), total=train_data.shape[0], desc="Processing rows"):
    generated_text = row['generated_text']
    pii_spans = row['pii_spans']
    pii_segments = extract_pii_segments(generated_text, pii_spans)

    prompt = generate_contextual_question_prompt(generated_text, pii_segments)
    while True:
        try:                
            chat_completion = client.chat.completions.create(
                    messages=[
                        {
                            "role": "user",
                            "content": prompt,
                        }
                    ],
                    model='Meta-Llama-3.1-8B-Instruct',
                )
            synthetic_text = chat_completion.choices[0].message.content
            qa.append(synthetic_text)
            break

        except Exception as e:
            print(f"Error encountered: {e}. Retrying in 1 minute...")
            time.sleep(60)

train_data['qa'] = qa


Processing rows:  30%|███       | 30/100 [00:08<00:18,  3.77it/s]

Error encountered: Error code: 429 - {'error': {'code': None, 'message': 'Rate limit exceeded', 'param': None, 'type': 'rate_limit_exceeded'}}. Retrying in 1 minute...


Processing rows:  62%|██████▏   | 62/100 [01:20<00:17,  2.21it/s]

Error encountered: Error code: 429 - {'error': {'code': None, 'message': 'Rate limit exceeded', 'param': None, 'type': 'rate_limit_exceeded'}}. Retrying in 1 minute...


Processing rows:  93%|█████████▎| 93/100 [02:34<00:01,  3.58it/s]

Error encountered: Error code: 429 - {'error': {'code': None, 'message': 'Rate limit exceeded', 'param': None, 'type': 'rate_limit_exceeded'}}. Retrying in 1 minute...


Processing rows: 100%|██████████| 100/100 [03:37<00:00,  2.18s/it]
/tmp/ipykernel_1532296/4153470807.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['qa'] = qa


In [33]:
import pandas as pd
import re

# Example data
data = train_data['qa']

# Initialize lists for storing parsed information
questions, option_a, option_b, option_c, option_d, correct_answers = [], [], [], [], [], []

# Process each question-answer block
for entry in data:
    # Extract the question
    question_match = re.search(r'^(.*?)\n\n', entry, re.DOTALL)
    question = question_match.group(1) if question_match else None
    questions.append(question)
    
    # Extract options
    options = re.findall(r'([A-D]\)) (.*?)(?=\n|$)', entry)
    option_a.append(options[0][1] if len(options) > 0 else None)
    option_b.append(options[1][1] if len(options) > 1 else None)
    option_c.append(options[2][1] if len(options) > 2 else None)
    option_d.append(options[3][1] if len(options) > 3 else None)
    
    # Extract correct answer
    correct_match = re.search(r'Correct answer: ([A-D]\)) (.*)', entry)
    correct_answer = correct_match.group(1) + " " + correct_match.group(2) if correct_match else None
    correct_answers.append(correct_answer)

# Create a DataFrame with the parsed data
df = pd.DataFrame({
    'question': questions,
    'option_a': option_a,
    'option_b': option_b,
    'option_c': option_c,
    'option_d': option_d,
    'correct_answer': correct_answers
})

# Display the DataFrame
print(df)

# Save to CSV (optional)
df.to_csv("processed_questions_general-4.csv", index=False)


                                             question  \
0   What is a common goal of employee well-being i...   
1   What type of coverage is included in the insur...   
2   What is the primary purpose of the "Supplement...   
3   What is a potential risk associated with inves...   
4   What is the primary purpose of a confidentiali...   
..                                                ...   
95  What is the primary purpose of the compliance ...   
96  What is the primary purpose of a bill of ladin...   
97  What is a key measure to enhance cybersecurity...   
98  What is a common reason why students may reque...   
99  What type of expenses are typically covered un...   

                                             option_a  \
0                 To increase employee turnover rates   
1                                                None   
2                    To report income from employment   
3   The bonds are guaranteed to increase in value ...   
4   To establish the scope of 